In [8]:
!pip install scikit-learn -U

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [9]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from scipy import signal
from sklearn.feature_selection import SequentialFeatureSelector

In [10]:
pickles_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/Pickles/'
A_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/Z/'
B_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/O/'
C_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/N/'
D_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/F/'
E_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/S/'

In [11]:
with open(pickles_address + 'As_allfeatures.pickle', 'rb') as f:
    As_features = pickle.load(f)
with open(pickles_address + 'Bs_allfeatures.pickle', 'rb') as f:
    Bs_features = pickle.load(f)
with open(pickles_address + 'Cs_allfeatures.pickle', 'rb') as f:
    Cs_features = pickle.load(f)
with open(pickles_address + 'Ds_allfeatures.pickle', 'rb') as f:
    Ds_features = pickle.load(f)
with open(pickles_address + 'Es_allfeatures.pickle', 'rb') as f:
    Es_features = pickle.load(f)
print('As_features', As_features.shape)
print('Bs_features', Bs_features.shape)
print('Cs_features', Cs_features.shape)
print('Ds_features', Ds_features.shape)
print('Es_features', Es_features.shape)

As_features (95, 100)
Bs_features (95, 100)
Cs_features (95, 100)
Ds_features (95, 100)
Es_features (95, 100)


In [12]:
def normalize_signals(inp):
    normalized_list = []
    for i in range(inp.shape[1]):
        sig = inp[:, i]
        for i in range(len(sig)):
            if str(sig[i]) == 'inf' or str(sig[i]) == '-inf':
                sig[i] = 0
        m = np.mean(sig)
        std = np.std(sig)
        normalized_list.append((sig - m) / std)
    return np.array(normalized_list).T

tp_a = normalize_signals(As_features)
tp_c = normalize_signals(Cs_features)

In [13]:
np.mean(tp_a[0]), np.std(tp_a[0])

(-0.2078768951179047, 0.018632683735243606)

In [20]:
X = list(tp_a.T) + list(tp_c.T)
y = [0] * len(tp_a.T) + [1] * len(tp_c.T)
len(X), len(y)

(200, 200)

In [21]:
final_features_size = 40
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean') 
# use different metrics
sfs = SequentialFeatureSelector(knn, n_features_to_select=final_features_size)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)
len(X_train), len(y_train), len(X_test), len(y_test)

(160, 160, 40, 40)

In [28]:
sfs.fit(X_train, y_train)
sfs.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [29]:
X_train = sfs.transform(X_train)
X_test = sfs.transform(X_test)

X_train.shape, X_test.shape

((160, 40), (40, 40))

In [30]:
knn.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean')

In [31]:
print(X_train[0])

[-0.15865703 -0.15856084 -0.15865643 -0.15865638 -0.15865641 -0.15855255
 -0.15865641 -0.15863735 -0.15865641 -0.15865637 -0.15865641 -0.15863735
 -0.15865641 -0.15865182 -0.15865641 -0.15865631 -0.15865641 -0.15865182
 -0.15865641 -0.15865351 -0.1586564  -0.15865612 -0.15865641 -0.15865351
 -0.15865641 -0.15865631 -0.15865641 -0.15865631 -0.15865641 -0.15865631
 -0.15810691  2.70354504 -0.15865635 -0.1586563  -0.1586562  -0.15839993
 -0.15865637 -0.13254081 -0.09227358 -0.15763289]


In [32]:
y_pred = knn.predict(X_test)

In [33]:
confusion_matrix(y_test, y_pred)

array([[21,  2],
       [ 0, 17]])

In [34]:
correctness = [1 if y_test[i] == y_pred[i] else 0 for i in range(len(y_pred))]
print('Accuracy:', (sum(correctness) / len(correctness)) * 100)

Accuracy: 95.0


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95        23
           1       0.89      1.00      0.94        17

    accuracy                           0.95        40
   macro avg       0.95      0.96      0.95        40
weighted avg       0.96      0.95      0.95        40



In [ ]:
!pip install stable_baselines3

In [47]:
from sklearn.preprocessing import MinMaxScaler
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [48]:
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [49]:
print(X_train[0])

[ 0.70186499  0.70185053  0.70183103  0.70183188  0.70183141  0.69992284
  0.70183141  0.70193816  0.70183139  0.70183221  0.70183141  0.70193816
  0.70183142  0.70177511  0.70183142  0.70183267  0.70183141  0.70177511
  0.70183143  0.70151717  0.70183153  0.70183637  0.70183141  0.70151717
  0.70183142  0.70182376  0.70183214  0.70181761  0.70183141  0.70182376
  0.70276764  0.22874013  0.70183142  0.70183228  0.70183083  0.70211027
  0.70183106 -0.9287362  -0.97068999 -0.28662877]


In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(2)
        self.observation_space = Box(low=np.array([-1]), high=np.array([1]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state